In [361]:
from aima3.logic import *
from aima3.utils import *

### exo 01

In [362]:
kb = FolKB()

kb.tell(expr('Homme(Jean)'))
kb.tell(expr('Homme(Fabian)')) 
kb.tell(expr('Homme(Franck)'))
kb.tell(expr('Homme(Jerome)'))
kb.tell(expr('Homme(Bruno)'))
kb.tell(expr('Femme(Evelyne)'))
kb.tell(expr('Femme(Louise)'))
kb.tell(expr('Femme(Marie)'))
kb.tell(expr('Femme(Eve)'))
kb.tell(expr('Femme(Sophie)'))

kb.tell(expr('Parent(Jean, Fabian)'))
kb.tell(expr('Parent(Fabian , Louise)'))
kb.tell(expr('Parent(Louise, Jerome)'))
kb.tell(expr('Parent(Louise, Sophie)'))
kb.tell(expr('Parent(Louise, Marie)'))
kb.tell(expr('Parent(Jerome, Franck)'))
kb.tell(expr('Parent(Sophie, Bruno)'))
kb.tell(expr('Parent(Sophie, Marie)'))
kb.tell(expr('Parent(Sophie, Eve)'))

In [363]:
kb.tell(expr(' Parent(x,y) & Femme(x) ==> Mere(x,y)'))
kb.tell(expr(' Parent(x,y) & Homme(x) ==> Pere(x,y)'))
kb.tell(expr(' Parent(z,x) & Parent(z,y) ==> FrereSoeur(x,y)'))
kb.tell(expr(' FrereSoeur(x,y) & Homme(x) ==> Frere(x,y)'))
kb.tell(expr('FrereSoeur(x,y) & Femme(x) ==> Soeur(x,y)'))
kb.tell(expr(' Parent(z,y) & Frere(z,x) ==> Oncle(z,x)'))
kb.tell(expr('Parent(z,y) & Soeur(z,x) ==> Tante(x,y)'))
kb.tell(expr(' Parent(z,y) & Oncle(z,x) ==> Cousin(x,y)'))
kb.tell(expr('Parent(x,y) ==> Ancetre(x,y)'))
kb.tell(expr('Parent(x,z) & Ancetre(z,y) ==> Ancetre(x,y)'))


In [364]:
print(bool(kb.ask(expr('Ancetre(Jean, Franck)'))))

True


### exo 2 with backtracking (not what the exo wants)

In [365]:
from aima3.csp import CSP, backtracking_search

regions = [1,2,3,4,5,6]

colors = ['Bleue','Vert','Blanc','Noir']

values_can_be_taken = { v : colors for v in range(1,7)}

adjacent_regions = {
    1:[3,2,5,6],
    2:[1,3,4,5,6],
    3:[1,2,4,6],
    4:[2,3],
    5:[1,2,6],
    6:[2,3,5,1]
}


def condition(A,a,B,b):
    return a != b

In [366]:
csp = CSP(regions,values_can_be_taken,adjacent_regions,condition)

result = backtracking_search(csp)

result

{1: 'Bleue', 2: 'Vert', 3: 'Blanc', 4: 'Bleue', 5: 'Blanc', 6: 'Noir'}

### exo 2 but with logic based solution

In [367]:
kb = FolKB()

colors = ['Bleue', 'Vert', 'Blanc', 'Noir']

adjacent_regions = {
    1: [3, 2, 5, 6],
    2: [1, 3, 4, 5, 6],
    3: [1, 2, 4, 6],
    4: [2, 3],
    5: [1, 2, 6],
    6: [2, 3, 5, 1]
}



In [368]:
for region, adjacents in adjacent_regions.items():
    for adj in adjacents:
        kb.tell(expr(f'Adjacent({region}, {adj})'))

kb.tell(expr('Color(x, c) & Color(y, c) & Adjacent(x, y) ==> Conflict()'))



def can_be_effected(region, color):

    kb.tell(expr(f'Color({region}, {color})'))
    conflict = kb.ask(expr('Conflict()'))
    kb.retract(expr(f'Color({region}, {color})'))

    return not conflict 


def solution() :

    coloration = {}

    for region , adjacents in adjacent_regions.items() :
        for color in colors :
            if can_be_effected(region,color) :
                kb.tell(expr(f'Color({region}, {color})'))
                coloration[region] = color
                break

    return coloration


In [369]:
coloring = solution()

print("Coloring:", coloring)



Coloring: {1: 'Bleue', 2: 'Vert', 3: 'Blanc', 4: 'Bleue', 5: 'Blanc', 6: 'Noir'}


In [370]:
result == coloring  #if both methodes gave the same result

True

In [371]:

#used for debugging after having a problem , to ignore

# def show_results():
#     coloring = {}
#     for region in adjacent_regions:
#         for color in colors:
#             if kb.ask(expr(f'Color({region}, {color})')):
#                 coloring[region] = color
#                 break
#     return coloring

# coloring = show_results()
# print("Coloring:", coloring)
# print("Is region 1 colored Bleue?", kb.ask(expr('Color(1, Bleue)')))
# print("Is Bleue used between region 1 and 2?", bool(kb.ask(expr('Used(1, 2, Bleue)'))))
# print("Re-testing Used:", bool(kb.ask(expr('Adjacent(4, 2)'))) and bool(kb.ask(expr('Color(2, Bleue)'))))
# print("Are regions 4 and 2 adjacent?", kb.ask(expr('Adjacent(4, 2)')))
# print("Is Vert used in region 1's adjacents?", not_used(1, 'Vert', adjacent_regions[1]))

# def is_valid_coloring():
#     for region, adjacents in adjacent_regions.items():
#         region_color = None
#         for color in colors:
#             if kb.ask(expr(f'Color({region}, {color})')):
#                 region_color = color
#                 break
        
#         if region_color is None:
#             return False
        
#         for adj in adjacents:
#             for color in colors:
#                 if kb.ask(expr(f'Color({adj}, {color})')) and color == region_color:
#                     return False
    
#     return True

# print("Is the coloring valid?", is_valid_coloring())

### exo 03